In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from sklearn.utils import shuffle
import sklearn 
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# X_train = pd.read_csv("./X_train.csv")
# y_train = pd.read_csv("./y_train.csv")
# X_test = pd.read_csv('./X_test.csv')
X_train = pd.read_csv("./X_train_denoised.csv")
y_train = pd.read_csv("./y_train.csv")
X_test = pd.read_csv('./X_test.csv')

In [5]:
#feature Extraction
def change1(x):
    return np.mean(np.abs(np.diff(x)))

def change2(x):
    return np.mean(np.diff(np.abs(np.diff(x))))

def feature_extraction(df):
    feat = pd.DataFrame()
    for col in df.columns[3:]:
        feat[col + '_mean'] = df.groupby(['series_id'])[col].mean()
        feat[col + '_std'] = df.groupby(['series_id'])[col].std()
        feat[col + '_max'] = df.groupby(['series_id'])[col].max()
        feat[col + '_min'] = df.groupby(['series_id'])[col].min()
        feat[col + '_max_to_min'] = feat[col + '_max'] / feat[col + '_min']
        feat[col + '_mean_abs_change'] = df.groupby('series_id')[col].apply(change1)
        feat[col + '_mean_abs_change2'] = df.groupby('series_id')[col].apply(change2)
        
    return feat
    
    

In [6]:
def group_kfold(train_df,y,folds):
    """Generator that yiels train and test indexes."""
    folds = GroupKFold(n_splits=folds)
    f_t = []
    f_test = []
    for train_idx, test_idx in folds.split(train_df, groups=y['group_id'].values):
        f_t.append(train_idx)
        f_test.append(test_idx)
        
    return f_t,f_test



In [7]:
#Machine Learning Algorithm (MLA) Selection and Initialization
def MLA_selection(X_train, y_train, folds):
    Train = feature_extraction(X_train)
    t1,t2 = group_kfold(Train,y_train,folds)
    le = LabelEncoder()
    target = le.fit_transform(y_train['surface'])
    target = pd.DataFrame(target)
    MLA = [
        #Ensemble Methods
        ensemble.AdaBoostClassifier(),
        ensemble.BaggingClassifier(),
        ensemble.ExtraTreesClassifier(),
        ensemble.GradientBoostingClassifier(),
        ensemble.RandomForestClassifier(),

        #Gaussian Processes
        gaussian_process.GaussianProcessClassifier(),

        #GLM
        linear_model.LogisticRegressionCV(),
        linear_model.PassiveAggressiveClassifier(),
        linear_model.RidgeClassifierCV(),
        linear_model.SGDClassifier(),
        linear_model.Perceptron(),

        #Navies Bayes
        naive_bayes.BernoulliNB(),
        naive_bayes.GaussianNB(),

        #Nearest Neighbor
        neighbors.KNeighborsClassifier(),

        #SVM
        svm.SVC(),
        svm.LinearSVC(),

        #Trees    
        tree.DecisionTreeClassifier(),
        tree.ExtraTreeClassifier(),

        #Discriminant Analysis
        discriminant_analysis.LinearDiscriminantAnalysis(),
        discriminant_analysis.QuadraticDiscriminantAnalysis(),


        XGBClassifier()
#         XGBClassifier()
        ]


    MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy','MLA Train Accuracy Mean', 'MLA Test Accuracy', 'MLA Test Accuracy Mean','MLA Test Accuracy Std' ]
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        test = []
        train = []
        for i in tqdm(range(0,5)):
            X_train = Train.iloc[t1[i],:]
            y_train = target.iloc[t1[i],:]
            X_test = Train.iloc[t2[i],:]
            y_test = target.iloc[t2[i],:]
            #Scaling
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            X_train = pd.DataFrame(X_train)
            X_test = pd.DataFrame(X_test)

            alg.fit(X_train,y_train)
            test.append(metrics.accuracy_score(y_test,alg.predict(X_test)))

            train.append(metrics.accuracy_score(y_train,alg.predict(X_train)))
            

                
        
        MLA_compare.loc[row_index, 'MLA Train Accuracy'] = train
        MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = np.mean(train)
        MLA_compare.loc[row_index, 'MLA Test Accuracy'] = test
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = np.mean(test)
        MLA_compare.loc[row_index, 'MLA Test Accuracy Std'] = np.std(test)


        row_index+=1

    
    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    
    return MLA_compare

In [96]:
MLA_selection(X_train, y_train, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

[18:10:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 1/5 [00:05<00:20,  5.02s/it]

[18:10:35] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 40%|████      | 2/5 [00:09<00:14,  4.78s/it]

...

[18:10:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 80%|████████  | 4/5 [00:17<00:04,  4.53s/it]

[18:10:47] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 5/5 [00:22<00:00,  4.46s/it]


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9973761889143982, 0.9908136482939632, 0.996...",0.993635,"[0.48751642575558474, 0.37139107611548555, 0.4...",0.476381,0.0558823
20,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.49934296977660975, 0.37270341207349084, 0.4...",0.471669,0.0576054
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.4980289093298292, 0.34908136482939633, 0.42...",0.469048,0.0722208
2,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.5256241787122208, 0.3320209973753281, 0.424...",0.460394,0.0749429
1,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...","[0.9931124959002952, 0.9957349081364829, 0.994...",0.993635,"[0.507227332457293, 0.30708661417322836, 0.385...",0.431791,0.0754181
16,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.43626806833114323, 0.3320209973753281, 0.39...",0.407885,0.0426353
17,ExtraTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.4152431011826544, 0.2874015748031496, 0.363...",0.394239,0.0627825
15,LinearSVC,"{'C': 1.0, 'class_weight': None, 'dual': True,...","[0.6188914398163332, 0.6551837270341208, 0.668...",0.646394,"[0.4336399474375821, 0.34120734908136485, 0.30...",0.393211,0.0606288
6,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': None, '...","[0.6448015742866514, 0.6850393700787402, 0.689...",0.670541,"[0.43889618922470436, 0.3451443569553806, 0.30...",0.391637,0.0625872
18,LinearDiscriminantAnalysis,"{'covariance_estimator': None, 'n_components':...","[0.5998688094457199, 0.6322178477690289, 0.641...",0.617457,"[0.43889618922470436, 0.35170603674540685, 0.3...",0.382187,0.0533244


In [8]:
#denoised FFT
MLA_selection(X_train, y_train, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

[17:14:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 20%|██        | 1/5 [00:07<00:30,  7.63s/it]

[17:14:25] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 40%|████      | 2/5 [00:14<00:21,  7.33s/it]

...

[17:14:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 80%|████████  | 4/5 [00:28<00:07,  7.19s/it]

[17:14:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 5/5 [00:35<00:00,  7.18s/it]


,MLA Name,MLA Parameters,MLA Train Accuracy,MLA Train Accuracy Mean,MLA Test Accuracy,MLA Test Accuracy Mean,MLA Test Accuracy Std
2,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.5282522996057819, 0.2979002624671916, 0.428...",0.457769,0.0885261
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...","[0.9970482125286979, 0.9950787401574803, 0.995...",0.995079,"[0.4980289093298292, 0.3293963254593176, 0.457...",0.454341,0.068532
20,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.4980289093298292, 0.33858267716535434, 0.42...",0.451987,0.0638282
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.5006570302233903, 0.2874015748031496, 0.420...",0.448052,0.0883982
1,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...","[0.9950803542144966, 0.994750656167979, 0.9940...",0.994751,"[0.5111695137976346, 0.2992125984251969, 0.394...",0.435726,0.0800442
16,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.4244415243101183, 0.2874015748031496, 0.382...",0.396336,0.0645472
19,QuadraticDiscriminantAnalysis,"{'priors': None, 'reg_param': 0.0, 'store_cova...","[0.7395867497540177, 0.7477034120734908, 0.763...",0.735566,"[0.392904073587385, 0.30708661417322836, 0.311...",0.380861,0.0636463
14,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': ...","[0.7422105608396196, 0.770997375328084, 0.7650...",0.761484,"[0.4323258869908016, 0.2335958005249344, 0.327...",0.370106,0.0792892
13,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...","[0.8094457199081666, 0.8270997375328084, 0.805...",0.813648,"[0.4126149802890933, 0.27165354330708663, 0.33...",0.363014,0.0626538
17,ExtraTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'crit...","[1.0, 1.0, 1.0, 1.0, 1.0]",1,"[0.41655716162943496, 0.18766404199475065, 0.3...",0.35933,0.0880142
